### Importing all the necessary Libraries 

In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, DoubleType, TimestampType

In [4]:
from datetime import datetime

In [5]:
from pyspark.sql.functions import *

In [6]:
from pyspark.sql import functions as f

In [7]:
spark

In [8]:
!cp 'Part1_Crime_data.csv' crime.csv

## Specifying the schema


In [9]:
schema = StructType([ \
    StructField("X",DoubleType(), True), \
    StructField("Y",DoubleType(), True), \
    StructField("RowID",IntegerType(), True), \
    StructField("CrimeDateTime",StringType(), True), \
    StructField("CrimeCode",StringType(), True), \
    StructField("Location",StringType(), True), \
    StructField("Description",StringType(), True),\
    StructField("Inside_Outside",StringType(), True),\
    StructField("Weapon",StringType(), True),\
    StructField("Post",IntegerType(), True),\
    StructField("District",StringType(), True),\
    StructField("Neighborhood",StringType(), True),\
    StructField("Latitude",DoubleType(), True),\
    StructField("Longitude",DoubleType(), True),\
    StructField("GeoLocation",StringType(), True),\
    StructField("Premise",IntegerType(), True),\
    StructField("VRIName",StringType(), True),\
    StructField("Total_Incidents",IntegerType(), True),\
    StructField("Shape",StringType(), True)\
    
])

## Reading file using schema 

In [10]:
df1 = spark.read.csv("crime.csv",sep=",", header=True, nullValue='NA', schema=schema)

In [20]:
df1.show()

+----------------+----------------+-----+--------------------+---------+--------------------+--------------+--------------+-------+----+---------+--------------------+--------+---------+------------------+-------+------------+---------------+-----+
|               X|               Y|RowID|       CrimeDateTime|CrimeCode|            Location|   Description|Inside_Outside| Weapon|Post| District|        Neighborhood|Latitude|Longitude|       GeoLocation|Premise|     VRIName|Total_Incidents|Shape|
+----------------+----------------+-----+--------------------+---------+--------------------+--------------+--------------+-------+----+---------+--------------------+--------+---------+------------------+-------+------------+---------------+-----+
|1420074.13302107|594160.602354662|    1|2022/02/26 04:00:...|       4E|   200 W MONUMENT ST|COMMON ASSAULT|          null|   null| 124|  CENTRAL|        MOUNT VERNON| 39.2975| -76.6193|(39.2975,-76.6193)|   null|        null|              1| null|
|141

### Printing schema

In [21]:
df1.printSchema()

root
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- RowID: integer (nullable = true)
 |-- CrimeDateTime: string (nullable = true)
 |-- CrimeCode: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Inside_Outside: string (nullable = true)
 |-- Weapon: string (nullable = true)
 |-- Post: integer (nullable = true)
 |-- District: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- GeoLocation: string (nullable = true)
 |-- Premise: integer (nullable = true)
 |-- VRIName: string (nullable = true)
 |-- Total_Incidents: integer (nullable = true)
 |-- Shape: string (nullable = true)



### cache the dataframe 

In [23]:
df1=df1.cache()

In [24]:
#counting the number of rows 
df1.count()

513158

In [25]:
#looking at the first five rows of dataframe
df1.take(5)

[Row(X=1420074.13302107, Y=594160.602354662, RowID=1, CrimeDateTime='2022/02/26 04:00:00+00', CrimeCode='4E', Location='200 W MONUMENT ST', Description='COMMON ASSAULT', Inside_Outside=None, Weapon=None, Post=124, District='CENTRAL', Neighborhood='MOUNT VERNON', Latitude=39.2975, Longitude=-76.6193, GeoLocation='(39.2975,-76.6193)', Premise=None, VRIName=None, Total_Incidents=1, Shape=None),
 Row(X=1411374.22509631, Y=589791.383964529, RowID=2, CrimeDateTime='2022/02/26 01:26:30+00', CrimeCode='9S', Location='2100 FREDERICK AVE', Description='SHOOTING', Inside_Outside='Outside', Weapon='FIREARM', Post=835, District='SOUTHWEST', Neighborhood='CARROLLTON RIDGE', Latitude=39.2856, Longitude=-76.6501, GeoLocation='(39.2856,-76.6501)', Premise=None, VRIName='Tri-District', Total_Incidents=1, Shape=None),
 Row(X=1411401.16887136, Y=582761.775193539, RowID=3, CrimeDateTime='2022/02/26 10:22:00+00', CrimeCode='4E', Location='2000 GRINNALDS AVE', Description='COMMON ASSAULT', Inside_Outside=Non

In [26]:
#converting the datetime column into timestamp format
df2 = df1.withColumn('CrimeDateTime', to_timestamp('CrimeDateTime', 'yyyy/MM/dd hh:mm:ss+00'))

In [29]:
df2.printSchema()

root
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- RowID: integer (nullable = true)
 |-- CrimeDateTime: timestamp (nullable = true)
 |-- CrimeCode: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Inside_Outside: string (nullable = true)
 |-- Weapon: string (nullable = true)
 |-- Post: integer (nullable = true)
 |-- District: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- GeoLocation: string (nullable = true)
 |-- Premise: integer (nullable = true)
 |-- VRIName: string (nullable = true)
 |-- Total_Incidents: integer (nullable = true)
 |-- Shape: string (nullable = true)



### Counting the distinct crime codes and looking at them 

In [32]:
df2.select("CrimeCode").distinct().count()

85

In [33]:
df2.select("CrimeCode").distinct().show()


+---------+
|CrimeCode|
+---------+
|       3P|
|       3K|
|      3BJ|
|       1A|
|       3M|
|       5F|
|       4B|
|       3B|
|       7A|
|      3NF|
|      3EF|
|       3N|
|       5D|
|       6K|
|      3LO|
|      3AF|
|       7B|
|      3GO|
|     3AJF|
|      8GV|
+---------+
only showing top 20 rows



In [34]:
df2=df2.cache()

### Number of crimes with respect to the crime code

In [35]:
crime_code=df2.groupBy("crimeCode").count()

In [36]:
crime_order=crime_code.orderBy('count', ascending=False)

In [37]:
crime_order.show(truncate=False)

+---------+-----+
|crimeCode|count|
+---------+-----+
|4E       |91650|
|6D       |68427|
|5A       |43928|
|7A       |40274|
|6J       |27636|
|6G       |26858|
|6E       |24300|
|6C       |23227|
|4C       |22438|
|5D       |14971|
|3AF      |14739|
|4B       |14460|
|4A       |13226|
|3B       |10737|
|4D       |7232 |
|5B       |6475 |
|9S       |5443 |
|6F       |5081 |
|5C       |4917 |
|6B       |4106 |
+---------+-----+
only showing top 20 rows



### Neighborhood with most number of crimes

In [39]:
df2.groupBy(["Neighborhood"]).count().orderBy('count',ascending=False).show(1)

+------------+-----+
|Neighborhood|count|
+------------+-----+
|    DOWNTOWN|17799|
+------------+-----+
only showing top 1 row



In [40]:
# Extracting the year and month from the crimedatetime column
df_month = df2.withColumn('year_month', month(df2['CrimeDateTime']))\
              .withColumn("year_of_crime", year(df2["CrimeDateTime"])).cache()

In [42]:
# removing any null values 
df3=df_month.na.drop(subset=["year_of_crime","year_month"])

### The month of the year with most number of crimes

In [44]:
year_month=df3.groupBy(["year_month"]).count().orderBy("count",ascending=False).show(1)

+----------+-----+
|year_month|count|
+----------+-----+
|         8|17151|
+----------+-----+
only showing top 1 row



In [45]:
#removing any row which has null values
df_weapon=df1.na.drop(subset=["Weapon"])

In [46]:
# Different weapons used in crimes
df_weapon.select("Weapon").distinct().show()

+-------+
| Weapon|
+-------+
|  HANDS|
|  KNIFE|
|  OTHER|
|   FIRE|
|FIREARM|
+-------+



### The weapon that was used the most.

In [47]:
df_weapon.groupBy("Weapon").count().orderBy("count", ascending=False).show(1)

+-------+-----+
| Weapon|count|
+-------+-----+
|FIREARM|46139|
+-------+-----+
only showing top 1 row

